$y'' = f(x,y,y')$ for $a <= x <= b$, with $y(a) = \alpha$ and $y(b) = \beta$
- a,b : endpoints
- $\alpha, \beta$ : boundary conditions
- h = 0.25
- $h = \frac {(b-a)}{N+1}$

Given $y'' = 2y^{3}, -1 <= x <= 0, y(-1) = \frac{1}{2}, y(0) = \frac{1}{3}$

In [60]:
# Define variables based on the given info
a = -1
b = 0
N = (b-a)/0.25 + 1
alpha = 1/2
beta = 1/3
print('N:',N)
print('alpha:',alpha)
print('beta:',beta)

N: 5.0
alpha: 0.5
beta: 0.3333333333333333


In [61]:
import numpy as np
# define f: y'' = 2y^3
def f(y):
    f = 2*y**3
    return f

# partial derivative of f respect to y
def fy(y):
    fy = 6*y**2
    return fy

# partial derivative of f respect to y'
def fy_p(y):
    fyp = 0
    return fyp

In [62]:
# Nonlinear Finite-Difference Algorithm
M = 20 # maximum number of iterations
TOL = 0.01
# Set h, w0, w_Npone (w_N+1)
h = 0.25
w0 = alpha
w_Npone = beta
w = [w0]
N = int(N)
for i in range(1,N+1): # (i: 1,2,..,N)
    wi = alpha + i*((beta-alpha)/(b-a))*h
    w.append(wi)
k = 1
while k <= M:
    x = a + h
    t = (w[2] - alpha)/(2*h)
    a1 = 2 + h**2*fy(w[1])
    b1 = -1 + (h/2)*fy(w[1])
    d1 = -(2*w[1] - w[2] - alpha + (h**2)*f(w[1]))
    
    a_list = [a, a1]
    b_list = [b, b1]
    c_list = [None, None]
    d_list = [None, d1]
    
    for i in range(1,N): # (i: 1,2,..,N-1)
        x = a + i*h
        t = (w[i+1] - w[i-1])/(2*h)
        ai = 2 + h**2*fy(w[i])
        bi = -1 + (h/2)*fy_p(w[i])
        ci = -1 - (h/2)*fy_p(w[i])
        di = -(2*w[i] - w[i+1] - w[i-1] + (h**2)*f(w[i]))
        
        a_list.append(ai)
        b_list.append(bi)
        c_list.append(ci)
        d_list.append(di)
        
    x = b-h
    t = (beta - w[N-1])/(2*h)
    aN = 2 + h**2*fy(w[N])
    cN = -1 -(h/2)*fy_p(w[N])
    dN = -(2*w[N] - w[N-1] - beta + (h**2)*f(w[N]))
    
    l1 = a1
    u1 = b1/a1
    z1 = d1/l1
    
    l_list = [None, l1]
    u_list = [None, u1]
    z_list = [None, z1]
    
    for i in range(2,N): # i = 2,3,..,N+1
        li = a_list[i] - c_list[i]*u_list[i-1]
        l_list.append(li)
        
        ui = b_list[i]/l_list[i]
        u_list.append(ui)
        
        zi = (d_list[i] - c_list[i]*z_list[i-1])/l_list[i]
        z_list.append(zi)
        
    lN = a_list[N] - c_list[N]*u_list[N-1]
    zN = (d_list[N] - c_list[N]*z_list[N-1])/lN
    
    l_list.append(lN)
    z_list.append(zN)
    
    vN = zN
    wN = w[N] + vN
    
    w_out = [wN]
    
    v3 = z_list[3] - u_list[3]*vN
    w3 = w[3] + v3
    w_out.append(w3)
    
    v2 = z_list[2] - u_list[2]*v3
    w2 = w[2] + v2
    w_out.append(w2)
    
    v1 = z_list[1] - u_list[1]*v2
    w1 = w[1] + v1
    w_out.append(w1)
    w_out.append(w0)
       
    k += 1

In [63]:
w_out.reverse()
print("Comparing the results from the algorithm with the actual result y = 1/(x+3)")
print('wi:',w_out)
x = np.arange(-1,0.25,0.25)
y = 1/(x+3)
print('f(xi):',list(y))
print('absolute errors:', abs(np.array(w_out)-y))

Comparing the results from the algorithm with the actual result y = 1/(x+3)
wi: [0.5, 0.44256095086703956, 0.39203366349528734, 0.3516011142272944, 0.2770207787038445]
f(xi): [0.5, 0.4444444444444444, 0.4, 0.36363636363636365, 0.3333333333333333]
absolute errors: [0.         0.00188349 0.00796634 0.01203525 0.05631255]
